In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from root_numpy import root2array
from ROOT import TEfficiency
from collections import OrderedDict

In [ ]:
phmaxes = [ 90, 190, 410 ] #210
mytypes = [ 'p', 'e', 'nu' ]
filebase, filemiddle, fileend = 'output/','_reco_smallwindow_thres','.root'
type_phmax_file = OrderedDict()
for phmax in phmaxes:
    for mytype in mytypes:
        type_phmax_file.update( { (mytype,phmax) : filebase + mytype + filemiddle + str(phmax) + fileend } )
  

In [ ]:
dfs = OrderedDict()
for mytypephmax, myfile in type_phmax_file.iteritems():
    dfs.update( { mytypephmax : pd.DataFrame( root2array( myfile, 'ana_tree' ) ) } )
print dfs.keys()

In [ ]:
k = TEfficiency()
k.SetStatisticOption(TEfficiency.kFFC)
def genEffsVsNRecoPE(mytype,myphmax,myspace=1,mynum_points=20):
    recoPE_threshs = np.linspace(0,mynum_points*myspace,mynum_points)
    effs, errors_upper, errors_lower = [], [], []
    for index in xrange(len(recoPE_threshs)):

        this_val = recoPE_threshs[index]   
        tmp_df = dfs[(mytype,myphmax)].query('%g < n_reco_PE and n_reco_PE < %g' % (this_val-myspace/2.,this_val+myspace/2.))
        denominator = len(tmp_df)
        tmp_df = tmp_df.query('n_trigs > 0')
        numerator = len(tmp_df)  
        if denominator: effs.append(float(numerator)/float(denominator))
        else: effs.append(0)
        errors_upper.append(k.FeldmanCousins(denominator,numerator,0.6827,True))
        errors_lower.append(k.FeldmanCousins(denominator,numerator,0.6827,False))
    return recoPE_threshs, effs, errors_upper, errors_lower

In [ ]:
def plotAllEffsVsNRecoPE(myxlabel='Reconstructed PE: OpHit PE within [-4us --> 10us] of True Interaction Time',myscale=1,myspace=1,mynum_points=20):
    plt.figure(figsize=(10,6))
    for mytype, myphmax in dfs.keys():
        xs, ys, eup, elow = genEffsVsNRecoPE(mytype,myphmax,myspace,mynum_points)
        xs *= myscale
        elow = np.array(ys) - np.array(elow)
        eup = np.array(eup) - np.array(ys)
        plt.errorbar(xs,ys,yerr=[elow,eup],fmt='o--',label="%s: PEthres %0.1f"%(mytype,myphmax/20.))
        plt.ylim((0,1))
        plt.xlabel(myxlabel)
        plt.ylabel('Trigger Efficiency')
        plt.title('Trigger Efficiency')
        plt.legend(loc=4)
        plt.grid(True)

In [ ]:
plotAllEffsVsNRecoPE(myxlabel='Summed OpHit PE within [-4us --> 10us] of True Interaction Time',myspace=8)

In [ ]:
def genEffsVsE(mytype, myphmax):
    space = .0025
    num_points = 40
    threshs = np.linspace(0,num_points*space,num_points)
    effs, errors_upper, errors_lower = [], [], []
    for index in xrange(len(threshs)):
        this_val = threshs[index]  
        mass = 0     
        if mytype == 'e': mass = 0.511/1000. 
        elif mytype == 'p': mass = 0.938
            
        myquery = 'energy - %f > %g and energy - %f < %g' % (mass, this_val-space/2.,mass,this_val+space/2.)
         
        tmp_df = dfs[(mytype,myphmax)].query(myquery)
        denominator = len(tmp_df)
        tmp_df = tmp_df.query('n_trigs > 0')
        numerator = len(tmp_df)
    
        if denominator: effs.append(float(numerator)/float(denominator))
        else: effs.append(0)
        errors_upper.append(k.FeldmanCousins(denominator,numerator,0.6827,True))
        errors_lower.append(k.FeldmanCousins(denominator,numerator,0.6827,False))
    return threshs, effs, errors_upper, errors_lower

def plotAllEffsVsE():
    plt.figure(figsize=(10,6))
    for myphmax in phmaxes:
        for mytype in ['p']:#,'e']:
            xs, ys, eup, elow = genEffsVsE(mytype, myphmax)
            elow = np.array(ys) - np.array(elow)
            eup = np.array(eup) - np.array(ys)
            plt.errorbar(xs,ys,yerr=[elow,eup],fmt='o--',label="%s: PEthres %0.1f"%(mytype,myphmax/20.))
            plt.ylim((0,1))
            plt.xlabel('Kinetic Energy of Interacting Particle [GEV]')
            plt.ylabel('Trigger Efficiency')
            plt.title('Trigger Efficiency vs. Single Particle Kinetic Energy [Single Protons]')
            plt.legend(loc=4)
            plt.grid(True)

In [ ]:
plotAllEffsVsE()